# mBuild Tutorial 00: Getting Started

This tutorial is designed to introduce you to the primary mission of mBuild - to facilitate the creation of complex molecular systems in a flexible fashion that allows system chemistry to be easily modulated. You will learn about the hierarchical data structure of mBuild `Compounds` and will have the ability to play around with some pre-built mBuild classes to observe how system structure can be changed by providing a few simple top-level variables.

### Hierarchical design approach

mBuild uses a [composite design pattern](https://en.wikipedia.org/wiki/Composite_pattern) to approach the creation of complex molecular systems. This yields the following features:
* Molecules feature a tree-like hierarchy (as shown in the figure below)
* All components in the hierarchy feature a common data structure (an mBuild `Compound`)
* The lowest level of the hierarchy (the 'leaves') are referred to as `Particles` and are typically individual atoms
* Atomic positions are maintained only at the `Particle` level; higher level components can compute properties based on contained `Particles`

Below is an example of an mBuild molecule hierarchy for an alkylsilane monolayer attached to a crystalline silica surface.
<img src="utils/hierarchical_design_image.png" alt="Drawing" style="width: 700px;"/>

### Primer on using Jupyter notebooks

[Jupyter notebooks](https://jupyter-notebook.readthedocs.io/en/stable/) provide an interactive environment for "developing, documenting, and executing code". Several languages are supported; however here we will be using Python. 

Jupyter notebooks feature two primary types of cells:
1. Markdown cells, like this cell, which contain explanatory text
2. Code cells, that can be executed by either clicking on the "run cell" icon or by hitting SHIFT + ENTER.

Cells do not have to be executed in order (however the cells in this tutorial are designed to be executed sequentially), and the order in which cells have been executed is recorded by the bracketed number to the left of the code cell (e.g. [ 1 ]). When a cell is executed you will first see an asterisk (i.e. [ * ]) which means that the cell is still running. When the asterisk is replaced by a number this means the execution has completed.

### Importing mBuild

To begin using mBuild we need to import the mBuild package, which is available through both the [Anaconda](https://anaconda.org/mosdef/mbuild) and [pip](https://pypi.python.org/pypi/mbuild/0.7.2) package managers. mBuild can also be downloaded from source, which is hosted on [Github](https://github.com/mosdef-hub/mbuild).

For the sake of cleaner output, we will also filter out some warnings that are raised by these tools.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import mbuild as mb

## Examples

Here, we'll explore some examples of mBuild `Compound` classes that have been pre-defined. Each features several top-level variables that can be manipulated to tune the system of interest.

### Alkanes

The `Alkane` recipe class allows one to create an alkane chain with a chain length (in number of carbons) specified by the `n` argument.

First, we will access the `Alkane` class and use Python's built-in `help` function to inspect the class constructor, which will reveal the arguments we are able to provide upon instantiation to customize our `Compound`.

In [2]:
help(mb.recipes.Alkane.__init__)

Help on function __init__ in module mbuild.lib.recipes.alkane:

__init__(self, n=3, cap_front=True, cap_end=True)
    Initialize an Alkane Compound.
    
    Args
    ----
    n: int, default 3,
        Number of carbon atoms.
    cap_front: bool, default True
        Whether to add a methyl group to the beginning of chain
        ('down' port).
    cap_end: bool, default True,
        Whether to add a methyl group to the end of chain ('up' port).



We see that there are three arguments we can provide when creating an `Alkane`, specifying the chain length and also whether or not we want to cap either end of the chain with a methyl group. Note that each of these arguments possesses a default value, and instantiating an `Alkane` without additional arguments will yield a propane chain.

However, here we'll pass `n=4` to our `Alkane` creation to yield an alkane with four carbons, i.e. butane.

The `visualize` method can be used to visualize our molecule interactively within the notebook.

In [3]:
butane = mb.recipes.Alkane(n=4)
butane.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

We are not restricted to just creating butane by using the `Alkane` class, however. By having the chain length as a tunable argument, we can easily create chains of any length. Here we'll simply change `n=4` to `n=10` to yield a decane molecule.

In [4]:
decane = mb.recipes.Alkane(n=10)
decane.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

By providing arguments when instantiating a `Compound` that allow us to tune the underlying chemistry, it becomes easy to create systems that span wide structural parameter spaces through altering a few top-level variables. This allows mBuild to become a valuable tool for screening-type studies.

We'll demonstrate this behavior once more by passing the `cap_end=False` argument to the instatiation of our decane chain. This will leave a `Port` on one end of the chain. You will learn more about `Port`s in a later tutorial, but briefly, these are what allow us to connect `Compound`s together. By leaving an open `Port` on the end of the alkane chain, we could now, e.g. make copies of the chain and functionalize a surface with them (as you will see in the following examples).

In [5]:
decane = mb.recipes.Alkane(n=10, cap_end=False)
decane.visualize(show_ports=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### Alkane monolayers

The `AlkaneMonolayer` class constructs an alkane monolayer on crystalline silica.  The `pattern` argument can be used to tune where chains are placed on the surface. The `tile_x` and `tile_y` arguments can be used to replicate the surface in the $x$ and $y$ dimensions. And the `chain_length` argument can be used to tune the length of the alkane chains.

First, we'll create the class

In [6]:
import mbuild as mb

from mbuild.lib.surfaces import Betacristobalite
from mbuild.lib.atoms import H


class AlkaneMonolayer(mb.recipes.Monolayer):
    """An alkane monolayer on beta-cristobalite. """

    def __init__(self, pattern, tile_x=1, tile_y=1, chain_length=10):
        """Create an alkane monolayer on beta-cristobalite.
        Parameters
        ----------
        pattern : np.ndarray, shape=(n, 3), optional, default=None
            An array of planar binding locations. If not provided, the entire
            surface will be filled with `chain`.
        tile_x : int, optional, default=1
            Number of times to replicate substrate in x-direction.
        tile_y : int, optional, default=1
            Number of times to replicate substrate in y-direction.
        chain_length : int, optional, default=10
            Number of carbon atoms per chain.
        """
        surface = Betacristobalite()
        alkane = mb.recipes.Alkane(chain_length, cap_end=False)
        hydrogen = H()
        super(AlkaneMonolayer, self).__init__(surface, alkane, backfill=hydrogen,
                                              pattern=pattern, tile_x=tile_x,
                                              tile_y=tile_y)

In [7]:
help(AlkaneMonolayer.__init__)

Help on function __init__ in module __main__:

__init__(self, pattern, tile_x=1, tile_y=1, chain_length=10)
    Create an alkane monolayer on beta-cristobalite.
    Parameters
    ----------
    pattern : np.ndarray, shape=(n, 3), optional, default=None
        An array of planar binding locations. If not provided, the entire
        surface will be filled with `chain`.
    tile_x : int, optional, default=1
        Number of times to replicate substrate in x-direction.
    tile_y : int, optional, default=1
        Number of times to replicate substrate in y-direction.
    chain_length : int, optional, default=10
        Number of carbon atoms per chain.



Now, we'll create a couple of different systems by varying the arguments we provide upon instantiation.

In [8]:
monolayer = AlkaneMonolayer(pattern=mb.Grid2DPattern(n=5, m=5), tile_x=1, tile_y=1, chain_length=20)
monolayer.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [9]:
# NBVAL_SKIP
monolayer = AlkaneMonolayer(pattern=mb.Random2DPattern(n=40), tile_x=1, tile_y=2, chain_length=5)
monolayer.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Recap

The goal of this tutorial was to demonstrate how with mBuild classes the chemistry of complex molecular systems can be easily tuned. In the following lessons, you will learn how to construct these classes so that you can begin using mBuild in your own research!